In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2


import matplotlib.cm as cm
from matplotlib import gridspec

import scipy
import scipy.spatial
from scipy import signal
from scipy import ndimage
from scipy.optimize import linear_sum_assignment

import numpy as np
import os
import cv2
from tqdm import trange

import h5py

import sleap

Autosaving every 180 seconds


In [2]:
# import functions

from sleap_correct_utility import Visualize, Track, Predict

In [22]:
#############################################  
############ LOAD TRACK DATA ################
#############################################
fname_slp = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.slp'
track = Track(fname_slp)

# 
track.load_tracks()

#
track.get_scores()

#track.plot_scores_distribution()



In [48]:
##############################################
### MAKE SPATIO-TEMP CONTINUOUS TRACKS #######
##############################################
# function that joins predictions by time and breaks them by space as required;
from sleap_correct_utility import Track
fname_slp = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.slp'
track = Track(fname_slp)

# 
track.load_tracks()
track.get_scores()

#
track.make_tracks_chunks()


#
vis = Visualize(track.tracks_spine)
# start = 850
# end = 1050

# start = 57800
# end = 58000

start = 3
end = 2000

vis.show_track_centers(track.tracks_spine,
                       start,
                       end)

# vis.show_track_distances(track.tracks_spine,
#                        start,
#                        end)

plt.show()


/home/cat/code/dlc_toolbox/sleap_correct_utility.py:501: RuntimeWarning: Mean of empty slice
  axis=2)


In [65]:
##############################################
### MAKE SPATIO-TEMP CONTINUOUS TRACKS #######
##############################################
print (track.scores.shape)
from sleap_correct_utility import Visualize

fname_video = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.avi'
fps=2
vis = Visualize(p)

# vis.make_video_centroid_scores(p.tracks_fixed,
#                                track.scores,
#                                fname_video,
#                                start,
#                                end,
#                                fps)

print (track.scores.shape, track.tracks_scores_mean.shape)

start = 700
end = 1150
start = int(8*60+30)*25
end = int(9*60+30)*25

vis.make_video_centroid_scores(track.tracks_spine,                # track locations
                               track.tracks_scores_mean,     # track confidence
                               track.scores,
                               fname_video,
                               start,
                               end,
                               fps) 


(89989, 4)
(89989, 4) (89989, 4)


100%|██████████| 1500/1500 [00:30<00:00, 49.36it/s]


In [108]:
# ##############################################
# ### MAKE SPATIO-TEMP CONTINUOUS TRACKS #######
# ##############################################


# animal_id = 3
# for c in range(len(track.tracks_chunks[animal_id])):
#     chunk = track.tracks_chunks[animal_id][c]
#     print ('chunk: ', c, " ", chunk[0], chunk[1],
#            " conf: ", track.tracks_spine_confidence[chunk[0]:chunk[1]+1,animal_id])
#     #print 

In [28]:
########################################################
####### RUN SINGLE FRAME MEMORY-BASED CORRECTION #######
########################################################
# initialize prediction 
verbose=False
p = Predict(track.tracks_spine,
            verbose)

# select start and ends
start = 57800
end = 58200

# get positions at time t
p.time = start
p.dynamics_off_flag=True  # don't take accel and ve into account

# optional
# p.delete_short_segs()

#
p.get_positions()
for t in trange(end-start):
    p.compute_vel_acceleration()
    p.predict_location_single_frame()
    p.compute_distances()
    p.update_positions()
    p.time+=1
    if p.time>=end:
        break


 17%|█▋        | 330/1997 [00:00<00:00, 3294.93it/s]

input into Predict:  (89989, 4, 2)


100%|█████████▉| 1996/1997 [00:00<00:00, 3020.29it/s]


In [38]:
#################################################################
####### RUN SPATIO-TEMPORAL TRACK MEMORY-BASED CORRECTION #######
#######       ALSO INCORPORATING CONFIDENCE VALUES        #######
#################################################################
# initialize prediction 
verbose=False
p = Predict(track.tracks_spine,
            verbose)

# select start and ends
start = 57800
end = 58200

# get positions at time t
p.time = start
p.dynamics_off_flag=True  # don't take accel and ve into account

# optional
# p.delete_short_segs()


track_selected = 3
print ("# of chunks in track ",track_selected, len(track.tracks_chunks[track_selected]))
print (track.tracks_spine_confidence.shape)


#
#p.get_chunk()
#for t in trange(end-start):
#    p.compute_vel_acceleration()
#    p.predict_location_single_frame()
#    p.compute_distances()
#    p.update_positions()
#    p.time+=1
#    if p.time>=end:
#        break



input into Predict:  (89989, 4, 2)
# of chunks in track  3 5341
(89989, 4)


In [90]:
print (track.tracks_chunks[animal_id][:5])
# loop over animal id
animal_id1 = 0
#for k in range(0,len(track.tracks_chunks[animal_id1])-1,1):
for k in range(0,20,1):
    # 
    times1 = track.tracks_chunks[animal_id1][k]
    #print (times)
    
    locs1 = track.tracks_spine[times1[1], animal_id1]
    #print (locs)
    
    #
    print ('looking to match end point: ', locs1)
    
    # search in time and spacefor nearest chunk 
    for animal_id2 in range(4):
        times2 = track.tracks_chunks[animal_id2]
        for time2 in times2:
            #print (p, time)
            if time2[0]>times1[1]:
                locs2 = track.tracks_spine[time2[0], animal_id2]
                
                dist = np.linalg.norm(locs1-locs2)
                
                print ("nearest animal: ", animal_id2, "  nearest time ", time2[0],
                      " locs: ", locs2, "  dist: ", dist)
                
                break
        
    
    
    
#     self.pos_read = self.tracks[self.time + 1]

#     if self.verbose:
#         print("self.pos_read at t+1:", self.pos_read)

#     # compute pairwise dist between predicted and read
#     cost = scipy.spatial.distance.cdist(self.pos_pred, self.pos_read)
#     if self.verbose:
#         print("Cost: ", cost)

#     # NEED TO SPEED THIS UP
#     idx = np.where(np.isnan(cost))
#     cost[idx] = 1E4

#     #
#     _, assignment = linear_sum_assignment(cost)
#     self.assignment = assignment

#     #
#     if self.verbose:
#         print("assignemnt: ", assignment)
#         print("costs: ", cost[_, assignment])
            


[[0, 3], [5, 8], [10, 44], [46, 134], [136, 139]]
looking to match end point:  [328.86740112 718.14868164]
nearest animal:  0   nearest time  5  locs:  [326.90576172 716.09124756]   dist:  2.842721258652345
nearest animal:  1   nearest time  379  locs:  [292.45251465 644.90118408]   dist:  81.79999911708366
nearest animal:  2   nearest time  524  locs:  [1208.64819336  413.27355957]   dist:  931.1085234522177
nearest animal:  3   nearest time  135  locs:  [467.96578979 602.26135254]   dist:  181.04760362237843
looking to match end point:  [331.67681885 718.22320557]
nearest animal:  0   nearest time  10  locs:  [334.11309814 717.09161377]   dist:  2.6862533028619127
nearest animal:  1   nearest time  379  locs:  [292.45251465 644.90118408]   dist:  83.15446394793246
nearest animal:  2   nearest time  524  locs:  [1208.64819336  413.27355957]   dist:  928.4789056872085
nearest animal:  3   nearest time  135  locs:  [467.96578979 602.26135254]   dist:  178.9464583594766
looking to match 

In [30]:
print (track.tracks_spine.shape)

(89989, 4, 2)


In [12]:
#################################  
####### VISUALIZE RESULTS #######
#################################
vis = Visualize(p.tracks)

#
ax=plt.subplot(2,1,1)
vis.show_track_centers(p.tracks,
                       start,
                       end)
plt.title("Pre-correction")

# visualize post-prediction
ax=plt.subplot(2,1,2)
vis.show_track_centers(p.tracks_fixed,
                       start,
                       end)
plt.title("Post-correction")
plt.show()



In [17]:
################################  
####### VISUALIZE SCORES #######
################################
print (track.scores.shape)
from sleap_correct_utility import Visualize

fname_video = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.avi'
fps=1
vis = Visualize(p)

# vis.make_video_centroid_scores(p.tracks_fixed,
#                                track.scores,
#                                fname_video,
#                                start,
#                                end,
#                                fps) 

vis.make_video_centroid_scores(track.tracks_spine,
                               track.scores,
                               fname_video,
                               start,
                               end,
                               fps) 


  0%|          | 0/400 [00:00<?, ?it/s]

(89989, 4)
(89989, 4)


100%|██████████| 400/400 [00:07<00:00, 50.29it/s]


In [5]:
print (track.scores.shape)
print (track.tracks.shape)
print (track.scores[0][0])
track.load_slp()

(89989, 4)
(89989, 4, 14, 2)
0.9999987


In [6]:
print (track.slp[0][0])

PredictedInstance(video=Video(filename=/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.avi, shape=(89989, 1024, 1280, 3), backend=MediaVideo), frame_idx=0, points=[nose: (406.8, 720.7, 0.91), lefteye: (386.4, 708.7, 1.02), righteye: (382.2, 740.3, 0.26), leftear: (361.9, 712.7, 1.00), rightear: (354.5, 747.9, 0.80), spine1: (354.8, 729.2, 0.86), spine2: (325.9, 716.5, 1.01), spine3: (302.3, 700.5, 1.04), spine4: (290.3, 676.4, 0.87)], score=1.00, track=Track(spawned_on=0, name='female'), tracking_score=0.00)


In [43]:
###########################  
####### MAKE VIDEOS #######
###########################
fname_video = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.avi'
fps=5
vis = Visualize(p)

vis.make_video_centroid(p.tracks_fixed,
                        fname_video,
                        start,
                        end,
                        fps)                        


  2%|▏         | 6/400 [00:00<00:07, 53.23it/s]

Histories:  (4, 5, 2)


100%|██████████| 400/400 [00:08<00:00, 49.97it/s]


In [50]:
fname_slp = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.slp'
track = Tracker(fname_slp)

# 
track.load_tracks()
track.get_scores()
track.show_scores()

/home/cat/miniconda3/envs/sleap2/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: Mean of empty slice


....loading slp file ...


100%|██████████| 89989/89989 [00:01<00:00, 82188.05it/s]


In [523]:
# FUNCTION TO TRY AND MASK OUT BLOCKS; 
# MOSTLY FAILED


class Video():
    
    def __init__(self, fname):
        
        self.fname = fname
        
        
    def init_vid(self, n):
        
        self.font = cv2.FONT_HERSHEY_PLAIN

        # load original vid
        self.original_vid = cv2.VideoCapture(self.fname)
        self.original_vid.set(cv2.CAP_PROP_POS_FRAMES, n)
        
    def init_save_vid(self, start, end):

        fps = 25
        size_vid = np.array([1280,1024])
        
        fname_out = self.fname[:-4]+"_"+str(start)+"_"+str(end)+'.mp4'

        fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
        self.video_out = cv2.VideoWriter(fname_out,
                                         fourcc, 
                                         fps, 
                                         (size_vid[0],size_vid[1]), 
                                         True)
        
    def save_vid(self, frame, surface, n):

        surface = (surface-np.min(surface))/(np.max(surface)-np.min(surface))*255
       
        self.surface[self.width:-self.width+1, 
                     self.width//2:-self.width//2+1,1] = surface.astype('uint8')
        
        frame = cv2.addWeighted(frame,0.0,
                                self.surface,1,0)

        cv2.putText(frame, str(n), (50, 50), self.font, 5, (255, 255, 0), 5)
            
            
        self.video_out.write(frame)

        
        
        
    def grab_frame(self):
        ret, frame = self.original_vid.read()
        
        return frame

    def show(self, frame):
        
        plt.imshow(frame)
        plt.show()
            
    
    def fit_mask(self, 
                frame):
        
        # print (frame.shape, mask.shape)
        
        
        frame_1d = frame[:,:,1]
        
        # find max for several rotations
        maxval=0
        surface = None
        for k in range(len(self.masks)):
            res = cv2.matchTemplate(frame_1d, 
                                    self.masks[k], 
                                    cv2.TM_CCOEFF)

            temp_maxval = np.amax(res)
            if temp_maxval>maxval:
                x,y = np.where(res==temp_maxval)
                maxval = temp_maxval
                surface=res
                #print ("new max found",temp_maxval, x,y)

        x=int(x+self.width)
        y=int(y+self.width)

        frame[x-self.width_mask:x+self.width_mask,
              y-self.width_mask:y+self.width_mask]=0
        
        

        return frame, x, y, surface
   

    def make_masks(self, mask):
        
        masks = []
        for k in range(-15,15,10):
            masks.append(ndimage.rotate(mask[:,:,1], k, reshape=False))
            
        self.masks=masks 
#    
fname = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.avi'
vid = Video(fname)

# mask size
x = 674
y = 614


# make rotated masks based on frame 0 block location
start=0
vid.init_vid(start)
frame = vid.grab_frame()

vid.width=32
vid.width_mask=64
mask = frame[x-vid.width:x+vid.width,
             y-vid.width//2:y+vid.width//2]
  
vid.make_masks(mask)

# restart 
start = 600
end = 650
vid.init_vid(start)
frame = vid.grab_frame()
vid.surface = np.zeros(frame.shape, 'uint8')

# initilize save vid
vid.init_save_vid(start,end)

for n in trange(start,end,1):
    
    # return masked frame and new x,y locs
    frame_masked, x, y, surface = vid.fit_mask(frame)
    
    # 
    vid.save_vid(frame_masked, surface, n)
    
    # 
    frame = vid.grab_frame()

vid.video_out.release()
        


100%|██████████| 50/50 [00:07<00:00,  6.32it/s]


In [512]:
print (z.shape)
ax=plt.subplot(1,2,1)
plt.imshow(surface)


ax=plt.subplot(1,2,2)
#plt.imshow((surface/np.max(surface)*255).astype('uint8'))
print (np.min(surface))

plt.show()

(897, 1153)
0.0


In [366]:
plt.imshow(mask)
plt.show()